In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler

# ==============================================================================
# EJERCICIO 3: ESTUDIO DETALLADO Y APLICACIÓN DE K-NN (K-Nearest Neighbors)
# ==============================================================================

"""
--- DOCUMENTACIÓN MEJORADA DEL ALGORITMO K-NN ---

1. ¿QUÉ ES K-NN?
   Es un algoritmo de aprendizaje supervisado "perezoso" (lazy learner).
   - Perezoso: No "aprende" un modelo matemático complejo (como pesos o coeficientes)
     durante el entrenamiento. Simplemente MEMORIZA todos los datos de entrenamiento.
   - Predicción: Cuando llega un dato nuevo, busca en su memoria los datos más parecidos.

2. PRINCIPIO DE FUNCIONAMIENTO:
   "Dime con quién andas y te diré quién eres".
   Para clasificar un punto nuevo 'P':
   A. Calcula la distancia entre 'P' y todos los puntos de entrenamiento almacenados.
      (Usualmente distancia Euclidiana: línea recta).
   B. Selecciona los 'K' puntos más cercanos (los K vecinos).
   C. Votación Mayoritaria: La clase más común entre esos K vecinos se asigna a 'P'.

3. PARÁMETROS CRÍTICOS:

   A. 'K' (Número de vecinos):
      - K = 1: El modelo es muy sensible al ruido. Si el vecino más cercano es una anomalía, fallará.
               Fronteras de decisión muy irregulares (Overfitting).
      - K alto: El modelo se vuelve muy suave. Si K es demasiado grande, la clase mayoritaria
                dominará siempre (Underfitting).
      - K suele ser un número impar (3, 5, 7) para evitar empates en la votación binaria.

   B. Métrica de Distancia:
      - Euclidiana (estándar).
      - Manhattan (suma de diferencias absolutas, útil en geometrías tipo cuadrícula).
      - Minkowski (generalización).

4. IMPORTANCIA DE LA ESCALA:
   Al igual que SVM, K-NN depende totalmente de calcular distancias.
   Es CRÍTICO normalizar los datos para que una variable con magnitud grande (ej: Salario)
   no opaque a una pequeña (ej: Edad).
"""

# ==============================================================================
# APLICACIÓN PRÁCTICA: DIAGNÓSTICO MÉDICO (BREAST CANCER)
# ==============================================================================

# 1. Carga de Datos
data = load_breast_cancer()
X = data.data
y = data.target
# 0: Maligno, 1: Benigno

print(f"Dataset Cáncer: {X.shape[0]} pacientes, {X.shape[1]} características.")

# 2. Preprocesamiento (CRÍTICO EN K-NN)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Búsqueda del mejor K (Aplicación de cambios para optimizar)
# En lugar de elegir K al azar, iteramos para encontrar el K óptimo usando validación.

k_values = range(1, 21)
accuracies = []

print("\n--- Buscando el valor óptimo de K ---")
for k in k_values:
    # Usamos cross_val_score para una evaluación más robusta en el set de entrenamiento
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='accuracy')
    accuracies.append(scores.mean())
    print(f"K={k}: Exactitud media = {scores.mean():.4f}")

# Graficar la curva de exactitud vs K
plt.figure(figsize=(10, 5))
plt.plot(k_values, accuracies, marker='o', linestyle='dashed', color='blue')
plt.title('Rendimiento de K-NN según número de vecinos (K)')
plt.xlabel('Valor de K')
plt.ylabel('Exactitud (Cross-Validation)')
plt.xticks(k_values)
plt.grid(True)
plt.show()

# Elegir el mejor K
best_k = k_values[np.argmax(accuracies)]
print(f"\nMejor K encontrado: {best_k}")

# 4. Entrenamiento y Evaluación Final con el mejor K
final_knn = KNeighborsClassifier(n_neighbors=best_k)
final_knn.fit(X_train_scaled, y_train)

y_pred = final_knn.predict(X_test_scaled)

print(f"\n--- RESULTADOS FINALES CON K={best_k} (Set de Prueba) ---")
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print(f"\nExactitud en Prueba: {accuracy_score(y_test, y_pred)*100:.2f}%")

print("\nCONCLUSIONES K-NN:")
print("1. K-NN es simple pero efectivo para este dataset médico.")
print("2. La gráfica muestra que un K muy bajo es inestable, y uno muy alto pierde precisión.")
print("3. La normalización de datos fue esencial para calcular distancias correctas.")